In [ ]:
# !pip install -qU selenium webdriver-manager persiantools python-docx

In [7]:
import csv
import re
import json
import logging
from tqdm import tqdm

import numpy as np
import pandas as pd

import docx

from selenium import webdriver

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36")

driver = webdriver.Chrome("chromedriver", options=options)

In [8]:
class JsonFileIterator:
    def __init__(self, path):
        self.path = path
        self.f = open(path, "r")
        self.i = 0
        self.length = self.counter_lines()

    def __iter__(self):
        return self

    def __next__(self):
        line = self.f.readline()
        if not line:
            # End of file
            self.f.close()
            raise StopIteration
        self.i += 1
        return json.loads(line)

    def counter_lines(self):
        with open(self.path, "r") as f1:
            return sum(1 for _ in f1)

    def __len__(self):
        return self.length

In [11]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.

    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)

In [16]:
search_query = pd.DataFrame(read_json_lines('./Torob/torob-search-data_v1.jsonl', n_lines=100000))
search_query.head()

,raw_query,result,clicked_result,clicked_rank,timestamp
0,لوستر سقفی برنز,"[7151290, 6462477, 7385791, 8451497, None, 269...",[9457219],[16],2022-07-24T09:21:58.752000+00:00
1,قیمت هلیکوپتر,"[363737, 3147253, 8720128, 9796388, 1420685, 5...","[3147253, 7135387]","[1, 7]",2022-07-24T07:32:12.261000+00:00
2,ساعت هوشمند,"[2459592, 9391819, 4229448, 7824893, 1670767, ...","[900897, 2931230, 31302]","[7, 25, 52]",2022-07-24T02:51:35.643000+00:00
3,تفلون مایع,"[5428407, 4474271, 4581189, 8504749, 4131340, ...","[7660686, 5901997, 2376830, 2383125]","[19, 18, 21, 36]",2022-07-24T02:51:54.771000+00:00
4,خط زن,"[8466568, 9285461, 1105044, 1314054, 1791930, ...",[3852139],[20],2022-07-23T20:56:36.043000+00:00


In [6]:
queries = dict()
for search in tqdm(search_query.to_dict('records')):
    query = search["raw_query"]
    if query not in queries:
        queries[query] = 1
    else:
        queries[query] += 1

100%|███████████████████████████| 2499901/2499901 [00:02<00:00, 1093824.73it/s]


In [7]:
sorted_queries = sorted(queries.items(), key=lambda x: x[1], reverse=False)

In [13]:
df = pd.DataFrame.from_dict(queries, orient='index', columns=['count'])
df.sort_values(by=['count'], inplace=True, ascending=True)

In [14]:
df

,count
Hq 282,1
سرسیلند ریو,1
لوله خرطومي,1
باتری لب تاپ دل ۵۱۱۰,1
تاپ و شلوارک ورزشی مردانه,1
...,...
گوشی شیایومی,9960
گوشی سامسونگ,12999
کیف مدرسه,16192
کوله پشتی مدرسه,17496


In [18]:
temp = df.copy(deep=True)
temp = temp[temp['count'] <= 10]
# temp[temp.index == "دچرخ"]

In [19]:
temp.sort_values(by=['count'], inplace=True, ascending=False)

In [20]:
temp

,count
فریزر خوابیده,10
سر دوشی حمام,10
گوشواره طلا میخی,10
Gt 1030,10
پتانسیومتر,10
...,...
اسكوپ بستني,1
پمپ اب ابگرمکن,1
گوشی موبایل شیامی ردمی 9A,1
ابمیوه گیر تفال,1


In [21]:
dfs = np.array_split(temp, len(temp)//50000 + 1)

In [ ]:
# dfs = []

# # loop over each output file and read it into a DataFrame
# for i in range(1, n_files + 1):
#     filename = f"output_{i}.csv"
#     df = pd.read_csv(filename, index_col=0)
#     dfs.append(df)

# # concatenate the DataFrames into a single DataFrame
# temp = pd.concat(dfs)

In [29]:
# !mkdir spell_check

In [33]:
for i, df in enumerate(dfs):
    df.to_csv(f"./spell_check/output_{i+1}.csv")

In [ ]:
# create an empty list to store the DataFrames
dfs = []

# loop over each output file and read it into a DataFrame
for i in range(1, n_files + 1):
    filename = f"output_{i}.csv"
    df = pd.read_csv(filename, index_col=0)
    dfs.append(df)

# concatenate the DataFrames into a single DataFrame
temp = pd.concat(dfs)


In [6]:
doc = docx.Document()

In [8]:
temp = temp[:100]

In [ ]:
# آب میوه گیری چهار

In [9]:
temp

,count
لگو کشتی دزد درییایی,1
کاوپوت زانتیا,1
دفتر مشق ۴۰برگ,1
گلسA52s,1
متور لگو,1
...,...
استتد,1
تلوزیون ۵۵ ایکس ویژن,1
ideapad 3 gaming,1
چاقویه همه کاره,1


In [10]:
table = doc.add_table(rows=1, cols=len(temp.columns))
header_cells = table.rows[0].cells
for i in range(len(temp.columns)):
    header_cells[i].text = temp.columns[i]

for i, (index, row) in tqdm(enumerate(temp.iterrows()), total=len(temp)):
    cells = table.add_row().cells
    cells[0].text = str(index)  # write the index value to the first cell
    for j in range(1, len(temp.columns)):
        cells[j].text = str(row[j]).replace('\n', ' ')
        
# save file
doc.save("output.docx")

100%|██████████| 100/100 [00:00<00:00, 423.73it/s]


In [3]:
df = pd.read_csv('./spell_check/output_5.csv', index_col=0)

In [4]:
temp = df.copy(deep=True)

In [5]:
temp

,count
لگو کشتی دزد درییایی,1
کاوپوت زانتیا,1
دفتر مشق ۴۰برگ,1
گلسA52s,1
متور لگو,1
...,...
اسكوپ بستني,1
پمپ اب ابگرمکن,1
گوشی موبایل شیامی ردمی 9A,1
ابمیوه گیر تفال,1


In [ ]:
with open("retrive_part_5.txt", "w", encoding="utf-8", newline="") as csvfile:

    wrtiter = csv.writer(csvfile)
    wrtiter.writerow(
        [
            "query",
            "spell_checked",
            "title_text",
        ]
    )

    for query in tqdm(temp.index):

        driver.get("https://torob.com/search/?query=" + query)

        check_spell = driver.find_elements("xpath", "//p[contains(@class, 'black_p')]")

        torob_spell_check = False
        corrected_form = None
        for text in check_spell:
            extracted_text = text.get_attribute("innerText")
            if extracted_text.startswith("جستجو غلط‌گیری شد:"):
                torob_spell_check = True
                corrected_query = re.search(
                    r"جستجو غلط‌گیری شد: (.*)", extracted_text
                ).group(1)
                corrected_form = corrected_query
            else:
                corrected_form = None

        names = driver.find_elements(
            "xpath", "//h2[contains(@class,'jsx-fa8eb4b3b47a1d18')]"
        )
        name_list = []
        for name in names:
            extracted_name = name.get_attribute("innerText")
            name_list.append(extracted_name)

        title_text = "\n".join(name_list)

        wrtiter.writerow(
            [
                query,
                corrected_form,
                title_text,
            ]
        )


  0%|▏                                  | 209/48357 [06:42<22:04:32,  1.65s/it]

In [1]:
import pandas as pd
# columns = ['query', 'spell_checked', 'title_text'] note that title_text is a text seprated with \n
df = pd.read_csv(f'retrive.csv', sep=',', encoding='utf-8')

ParserError: Error tokenizing data. C error: Expected 3 fields in line 172, saw 4


In [2]:
def parse_text_column(text):
    return text.split('\n')


In [5]:
def parse_text_column(text):
    return text.split('\n')

df = pd.read_csv('retrive.txt', sep=',', converters={'title_text': parse_text_column})


In [7]:
df

,query,spell_checked,title_text
0,قفل دیسکی,NaN,[قفل دیسکی آژیر دار CAP ا قفل دیسک موتور آژیرد...
1,ساعت ضد اب زنانه,NaN,[ساعت مچی ال ای دی بچگانه عروسکی ا Children Do...
2,cute,NaN,"[جامدادی طرح دختر کیوت, ست پک استیکر فانتزی ww..."
3,کبف مدرسه پسرانه,کیف مدرسه پسرانه,"[کیف مدرسه پسرانه طرح سگ های نگهبان مدل 46, کی..."


In [15]:
cd word_data_spell_checking

c:\Users\amkyp\Documents\UT - MIT\term_4012\torob_contest\word_data_spell_checking


In [21]:
import difflib
import docx

# def extract_text(filename):
#     doc = docx.Document(filename)
#     text = [p.text for p in doc.paragraphs]
#     return "\n".join(text)

def extract_text(filename):
    doc = docx.Document(filename)
    text = ""
    for table in doc.tables:
        for row_idx, row in enumerate(table.rows):
            if row_idx == 0:  # skip header row
                continue
            for cell in row.cells:
                text += cell.text + " "
            text += "\n"
    return text.strip()

# Extract text from both files
text1 = extract_text("output_1.docx")
text2 = extract_text("output_1_corrected.docx")


text1[:100], text2[:100]

('دوچرخه \nکوله پشتی مدرسه \nکیف مدرسه \nگوشی سامسونگ \nگوشی شیایومی \nکفش مردانه \nساعت هوشمند \nجاروبرقی \nد',
 'دوچرخه \nکوله پشتی مدرسه \nکیف مدرسه \nگوشی سامسونگ \nگوشی شیائومی \nکفش مردانه \nساعت هوشمند \nجاروبرقی \nد')

In [ ]:
import difflib
import docx
import pandas as pd

def extract_text(filename):
    doc = docx.Document(filename)
    text = ""
    for table in doc.tables:
        for row_idx, row in enumerate(table.rows):
            if row_idx == 0:  # skip header row
                continue
            for cell in row.cells:
                text += cell.text + " "
            text += "\n"
    return text.strip()

text1 = extract_text("output_1.docx")
text2 = extract_text("output_1_corrected.docx")

with open("text1", 'w') as f:
    f.write(text1)

with open('text2', 'w') as f:
    f.write(text2)

def parse_text_column(text):
    return text.split('\n')

df1 = pd.read_csv("./word_data_spell_checking/text1", converters={'title_text': parse_text_column}, header=None)
df2 = pd.read_csv("./word_data_spell_checking/text2", converters={'title_text': parse_text_column}, header=None)

correction = {}
for i, (index, row) in enumerate(df1.iterrows()):
    if row[0] != df2.iloc[i][0]:
        correction[row[0]] = df2.iloc[i][0]

In [36]:
with open("text1", 'w') as f:
    f.write(text1)

with open('text2', 'w') as f:
    f.write(text2)

In [2]:
import pandas as pd

In [3]:
def parse_text_column(text):
    return text.split('\n')

In [6]:
df1 = pd.read_csv("./word_data_spell_checking/text1", converters={'title_text': parse_text_column}, header=None)
df2 = pd.read_csv("./word_data_spell_checking/text2", converters={'title_text': parse_text_column}, header=None)

In [19]:
df1

,0
0,دوچرخه
1,کوله پشتی مدرسه
2,کیف مدرسه
3,گوشی سامسونگ
4,گوشی شیایومی
...,...
995,3080
996,میدرنج 8
997,سردوز
998,چراغ قوه پیشانی


In [9]:
import difflib


In [11]:
# get diff of df1 and df2 and make a dict of them as word(df):corrected(df2)
correction = {}
for i, (index, row) in enumerate(df1.iterrows()):
    if row[0] != df2.iloc[i][0]:
        correction[row[0]] = df2.iloc[i][0]

In [18]:
# show some items of the dict
# list(correction.items())[:10]
for item in list(correction.items())[:10]:
    print(item[0], item[1])

گوشی شیایومی  گوشی شیائومی 
اسباب بازی  اسباب‌بازی 
ساعت مچی مردانه  ساعت‌مچی مردانه 
ساعت مچی  ساعت‌مچی 
ایرپاد  ایر پاد 
ساعت هوشمند شیایومی  ساعت هوشمند شیائومی 
اسباب بازی دخترانه  اسباب‌بازی دخترانه 
عینک افتابی  عینک‌آفتابی 
پلی استیشن 4  پلی‌استیشن 4 
چراغ قوه  چراغ‌قوه 


In [26]:
import difflib
import docx
import pandas as pd
import os
import copy

def extract_text(filename):
    doc = docx.Document(filename)
    text = ""
    for table in doc.tables:
        for row_idx, row in enumerate(table.rows):
            if row_idx == 0:  # skip header row
                continue
            for cell in row.cells:
                text += cell.text + " "
            text += "\n"
    return text.strip()


In [17]:
len(correction)

1775

In [18]:
# correction = {}

for i in range(5, 12):
    filename1 = f"./word_data_spell_checking/output_{i}.docx"
    filename2 = f"./word_data_spell_checking/output_{i}_corrected.docx"

    text1 = extract_text(filename1)
    text2 = extract_text(filename2)

    with open(f"./word_data_spell_checking/text1_{i}.txt", 'w') as f:
        f.write(text1)

    with open(f"./word_data_spell_checking/text2_{i}.txt", 'w') as f:
        f.write(text2)

    def parse_text_column(text):
        return text.split('\n')

    df1 = pd.read_csv(f"./word_data_spell_checking/text1_{i}.txt", converters={'title_text': parse_text_column}, header=None)
    df2 = pd.read_csv(f"./word_data_spell_checking/text2_{i}.txt", converters={'title_text': parse_text_column}, header=None)

    for j, (index, row) in enumerate(df1.iterrows()):
        if row[0] != df2.iloc[j][0]:
            correction[row[0]] = df2.iloc[j][0]

    each_correction = copy.deepcopy(correction)

    with open(f"./word_data_spell_checking/correction_{i}.txt", 'w') as f:
        for key, value in each_correction.items():
            f.write(f"{key}: {value}\n")

    os.remove(f"./word_data_spell_checking/text1_{i}.txt")
    os.remove(f"./word_data_spell_checking/text2_{i}.txt")

NameError: name 'extract_text' is not defined

In [ ]:
# get all corrections and save them as one file in dict format
correction = {}
for i in range(1, 12):
    with open(f"./word_data_spell_checking/correction_{i}.txt", 'r') as f:
        for line in f.readlines():
            key, value = line.split(": ")
            correction[key] = value.strip()

In [ ]:
# store as a dict 
with open("./word_data_spell_checking/correction.txt", 'w') as f:
    for key, value in correction.items():
        f.write(f"{key}: {value}\n")

In [ ]:
import joblib
joblib.dump(correction, "./word_data_spell_checking/correction.pkl")

['./word_data_spell_checking/correction.pkl']

In [27]:
# search_query = search_query.set_index('raw_query')
search_query.reset_index()

,raw_query,result,clicked_result,clicked_rank,timestamp
0,لوستر سقفی برنز,"[7151290, 6462477, 7385791, 8451497, None, 269...",[9457219],[16],2022-07-24T09:21:58.752000+00:00
1,قیمت هلیکوپتر,"[363737, 3147253, 8720128, 9796388, 1420685, 5...","[3147253, 7135387]","[1, 7]",2022-07-24T07:32:12.261000+00:00
2,ساعت هوشمند,"[2459592, 9391819, 4229448, 7824893, 1670767, ...","[900897, 2931230, 31302]","[7, 25, 52]",2022-07-24T02:51:35.643000+00:00
3,تفلون مایع,"[5428407, 4474271, 4581189, 8504749, 4131340, ...","[7660686, 5901997, 2376830, 2383125]","[19, 18, 21, 36]",2022-07-24T02:51:54.771000+00:00
4,خط زن,"[8466568, 9285461, 1105044, 1314054, 1791930, ...",[3852139],[20],2022-07-23T20:56:36.043000+00:00
...,...,...,...,...,...
99995,مبل,"[4606753, 834291, 2278933, 1499424, 407025, 96...","[7818170, 5930038, 5930038, 1265547, 4744144, ...","[15, 60, 60, 129, 195, 232]",2022-07-25T19:14:39.382000+00:00
99996,ساعت مچی,"[None, 4257902, 9724490, 1229540, 1896213, 247...",[2366146],[21],2022-07-25T17:52:10.268000+00:00
99997,هواپیما جنگی کنترلی,"[1696410, 2843164, 232443, 2972477, 438993, 89...",[1696410],[0],2022-07-25T11:44:20.013000+00:00
99998,طبل,"[2520788, 6360931, 9652590, 7974759, 977328, 1...",[6360931],[1],2022-07-25T19:14:00.541000+00:00


In [29]:
# import re

# # Assume `df` is your DataFrame with a column named "query" containing the queries to be checked
# # Assume `correction` is your dictionary with misspelled words as keys and corrected words as values

# pattern = re.compile(r'\b(' + '|'.join(correction.keys()) + r')\b')

# def replace_words(match):
#     return correction[match.group(0)]

# df["query"] = df["query"].str.replace(pattern, replace_words)


In [30]:
pattern = re.compile(r'\b(' + '|'.join(correction.keys()) + r')\b')

In [48]:
def replace_words(match):
    print(match.group(0))
    return correction[match.group(0)]

In [49]:
search_query["raw_query"] = search_query["raw_query"].str.replace(pattern, replace_words)

# search_query.reset_index(inplace=True)
# search_query

ریلمی جی تی 
پلی استیشن 
ایفون 11پرو 


In [42]:
search_query[search_query["raw_query"] == "دوجرخه"]

,raw_query,result,clicked_result,clicked_rank,timestamp
6705,دوجرخه,"[5086505, 9431396, 1856490, 2455824, 5197810, ...",[5197810],[4],2022-07-24T13:33:06.788000+00:00
54945,دوجرخه,"[5086505, 9431396, 1856490, 2455824, 5197810, ...",[9431396],[1],2022-07-24T11:53:13.680000+00:00
71489,دوجرخه,"[5086505, 9431396, 1856490, 2455824, 5197810, ...","[1856490, 9431396, 2402666, 9179062, 8793174, ...","[2, 1, 11, 18, 22, 22]",2022-07-24T16:59:56.515000+00:00
92895,دوجرخه,"[5086505, 9431396, 1856490, 2455824, 5197810, ...","[9431396, 9431396, 9431396, 9431396, 9431396, ...","[1, 1, 1, 1, 1, 1]",2022-07-25T07:10:47.602000+00:00


In [22]:
# Find the differences between the texts
differences = list(difflib.ndiff(text1.splitlines(), text2.splitlines()))

In [25]:
list(difflib.ndiff(text1.splitlines(), text2.splitlines()))


['  دوچرخه ',
 '  کوله پشتی مدرسه ',
 '  کیف مدرسه ',
 '  گوشی سامسونگ ',
 '- گوشی شیایومی ',
 '?         ^\n',
 '+ گوشی شیائومی ',
 '?         ^\n',
 '  کفش مردانه ',
 '  ساعت هوشمند ',
 '  جاروبرقی ',
 '  دوچرخه اسپرت ',
 '  لباس مجلسی ',
 '  جامدادی ',
 '  صندلی گیمینگ ',
 '  کولر ',
 '  قمقمه ',
 '  کاپشن ',
 '  طبل محرم ',
 '  پارچه ',
 '  a13 ',
 '- اسباب بازی ',
 '?      ^\n',
 '+ اسباب\u200cبازی ',
 '?      ^\n',
 '  ساعت مردانه ',
 '  روغن ',
 '  خودرو ',
 '  کولر گازی ',
 '- ساعت مچی مردانه ',
 '?     ^\n',
 '+ ساعت\u200cمچی مردانه ',
 '?     ^\n',
 '  تفنگ شکاری ',
 '  پراید ',
 '  برنج ',
 '  ساعت ',
 '  ماساژور ',
 '  دریل ',
 '  مبل ',
 '- ساعت مچی ',
 '?     ^\n',
 '+ ساعت\u200cمچی ',
 '?     ^\n',
 '  طبل ',
 '  لوازم تحریر ',
 '  کفش زنانه ',
 '  تلویزیون سامسونگ ',
 '- ایرپاد ',
 '+ ایر پاد ',
 '?    +\n',
 '  ترازو ',
 '  پرچم محرم ',
 '  هدفون ',
 '  a32 ',
 '  چادر مسافرتی ',
 '  اسپیکر ',
 '  a23 ',
 '  پتو ',
 '  چاقو ',
 '  یخچال ',
 '  تابلو ',
 '  باربی ',
 ' 

In [23]:
# Create a dictionary of corrections
corrections = {}
for diff in differences:
    if diff.startswith("+ "):
        parts = diff[2:].split("\t")
        if len(parts) == 2:
            corrections[parts[0]] = parts[1]

In [24]:
# Print the corrections
for key, value in corrections.items():
    print(f"{key}: {value}")
